In [7]:
import tkinter as tk
import threading
import numpy as np
from scipy.signal import butter, lfilter, freqz
from scipy.io import wavfile
import matplotlib.pyplot as plt
import pyaudio

#### Import Track

In [8]:
sample_rate, audio_data = wavfile.read("120 BPM - ROCK - 4⧸4 Drum Track - Metronome - Drum Beat.wav")
audio_data = audio_data.T
print("sample_rate :",sample_rate)

sample_rate : 48000


In [9]:
def butter_lowpass_filter(data, cutoff_frequency, sampling_rate, order=4):
    nyquist = 0.5 * sampling_rate
    normal_cutoff = cutoff_frequency / nyquist
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    y = lfilter(b, a, data)
    return y

def butter_highpass_filter(data, cutoff_frequency, sampling_rate, order=4):
    nyquist = 0.5 * sampling_rate
    normal_cutoff = cutoff_frequency / nyquist
    b, a = butter(order, normal_cutoff, btype='high', analog=False)
    y = lfilter(b, a, data)
    return y

def butter_bandpass_filter(data, lowcut, highcut, sampling_rate, order=4):
    nyquist = 0.5 * sampling_rate
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band', analog=False)
    y = lfilter(b, a, data)
    return y

In [31]:
def update_gain(value, low = False, mid = False, high=False, fre=False):
    global lp_scale, bp_scale, hp_scale, lp_gain, bp_gain, hp_gain
    value = int(value)
    if low:
        lp_gain = 10**(value/20)
    elif high:
        hp_gain = 10**(value/20)
    elif mid:
        bp_gain = 10**(value/20)
    elif fre:
        mid_fre = value
    lp_scale = lp_gain/(lp_gain + bp_gain + hp_gain)
    bp_scale = bp_gain/(lp_gain + bp_gain + hp_gain)
    hp_scale = hp_gain/(lp_gain + bp_gain + hp_gain)

In [36]:
def create_play():
    thread = threading.Thread(target=play)
    thread.start()

def play():
    global audio_data, sample_rate
    #global deno_L, nume_L, deno_M, nume_M, deno_H, nume_H 
    global lp_scale, bp_scale, hp_scale, mid_fre
    CHUNK = sample_rate
    p = pyaudio.PyAudio()
    stream = p.open(format=pyaudio.paInt32,
                channels = 1,
                rate = sample_rate,
                frames_per_buffer = CHUNK,
                output = True)
    
    y_prev_lp = np.zeros(2)
    y_prev_bp = np.zeros(2)
    y_prev_hp = np.zeros(2)
    
    cutoff_frequency_lp = 100  # Hz
    cutoff_frequency_lp = 6500  # Hz
    for k in range(6*CHUNK, 46*CHUNK, CHUNK):
        x = np.array(audio_data[0, k : k+CHUNK])
        # Low-pass filter
        lpf_signal = butter_lowpass_filter(x, cutoff_frequency_lp, sample_rate)
        lpf_signal = lpf_signal.astype(int)
        lpf_signal = scale_array(x, lpf_signal)

        # High-pass filter
        hpf_signal = butter_highpass_filter(x, cutoff_frequency_lp, sample_rate)
        hpf_signal = hpf_signal.astype(int)
        hpf_signal = scale_array(x, hpf_signal)

        # Band-pass filter
        lowcut_bp = mid_fre - 50  # Hz
        highcut_bp = mid_fre + 50  # Hz
        bpf_signal = butter_bandpass_filter(x, lowcut_bp, highcut_bp, sample_rate)
        bpf_signal = bpf_signal.astype(int)
        bpf_signal = scale_array(x, bpf_signal)

        
        y = lp_scale*lpf_signal + bp_scale*0.5*bpf_signal + hp_scale*hpf_signal
        y = y.astype(int)

        #Reshaping
        y_n = y.reshape(1, -1)
        #Playing
        stream.write(y_n.tobytes()) 

In [ ]:
lp_gain = 1
bp_gain = 1
hp_gain = 1
mid_fre = 500

lp_scale = lp_gain/(lp_gain + bp_gain + hp_gain)
bp_scale = bp_gain/(lp_gain + bp_gain + hp_gain)
hp_scale = hp_gain/(lp_gain + bp_gain + hp_gain)


# Create the main window
root = tk.Tk()
root.title("EQ")

# Create a label to display the selected value
label = tk.Label(root, text="Equalizer")
label.pack(pady=10)

root.geometry("500x400")

# Bass vertical slider 
bass_slider = tk.Scale(root, from_=15, to=-15, orient=tk.VERTICAL, length=200, command=lambda value: update_gain(value, low=True))
bass_slider.place(x=50, y=50)
label_s = tk.Label(root, text="LF")
label_s.place(x=65, y=260)

# frequency slider 
frequency_slider = tk.Scale(root, from_=500, to=12000, orient=tk.VERTICAL, length=200, command=lambda value:update_gain(value, fre = True))
frequency_slider.place(x=350, y=50)
label_s = tk.Label(root, text="Fre")
label_s.place(x=380, y=260)

# Mid vertical slider 
mid_slider = tk.Scale(root, from_=15, to=-15, orient=tk.VERTICAL, length=200, command=lambda value:update_gain(value, mid = True))
mid_slider.place(x=150, y=50)
label_s = tk.Label(root, text="MF")
label_s.place(x=170, y=260)

# Treble vertical slider 
treble_slider = tk.Scale(root, from_=15, to=-15, orient=tk.VERTICAL, length=200, command=lambda value:update_gain(value, high = True))
treble_slider.place(x=250, y=50)
label_s = tk.Label(root, text="HF")
label_s.place(x=270, y=260)

#play/pause
play_button = tk.Button(text = 'Play/Pause', command = create_play)
play_button.place(x=200, y=360)

# Run the Tkinter event loop
root.mainloop()